In [12]:
import requests
from bs4 import BeautifulSoup
# Make a request to the website
r = requests.get('https://www.bbc.com')
# Create an object to parse the HTML format
soup = BeautifulSoup(r.content, 'html.parser')
# Retrieve all popular news links (Fig. 1)
link = []
for i in soup.find('div', {'class':"module__content"}).find_all('a'):
    i['href'] = "https://www.bbc.com/"+i['href'] + '?page=all'
    link.append(i['href'])

# For each link, we retrieve paragraphs from it, combine each paragraph as one string, and save it to documents (Fig. 2)
documents = []
for i in link:
    # Make a request to the link
    r = requests.get(i)
  
    # Initialize BeautifulSoup object to parse the content 
    soup = BeautifulSoup(r.content, 'html.parser')
  
    # Retrieve all paragraphs and combine it as one
    sen = []
    for i in soup.find('div', {'class':'ssrcss-7uxr49-RichTextContainer e5tfeyi1'}).find_all('p'):
        sen.append(i.text)
  
    # Add the combined paragraphs to documents
    documents.append(' '.join(sen))
    print(documents)

['Chinese people have rushed to book overseas travel after Beijing announced it would reopen its borders next month.']


AttributeError: 'NoneType' object has no attribute 'find_all'

In [27]:
import re
import string
documents_clean = []
for d in documents:
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)

In [28]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()
# It fits the data and transform it as a vector
X = vectorizer.fit_transform(documents_clean)
# Convert the X as transposed matrix
X = X.T.toarray()
# Create a DataFrame and set the vocabulary as the index
df = pd.DataFrame(X, index=vectorizer.get_feature_names())
df

,0,1,2,3,4,5,6,7,8,9
abimanyu,0.067871,0.028643,0.026406,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
absen,0.000000,0.000000,0.000000,0.038035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ada,0.024495,0.000000,0.000000,0.020418,0.026721,0.023005,0.025800,0.000000,0.000000,0.029080
adalah,0.044485,0.000000,0.017307,0.000000,0.024264,0.020889,0.000000,0.014957,0.030402,0.026406
adanya,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054170
...,...,...,...,...,...,...,...,...,...,...
yang,0.185570,0.028477,0.039380,0.098436,0.018403,0.110907,0.017769,0.011344,0.034588,0.100139
year,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030683,0.000000,0.000000
yong,0.073485,0.144721,0.247777,0.000000,0.160328,0.000000,0.000000,0.000000,0.167404,0.058160
yunani,0.000000,0.000000,0.000000,0.000000,0.000000,0.042854,0.000000,0.000000,0.000000,0.000000


In [32]:
def get_similar_articles(q, df):
  import numpy as np
  print("query:", q)
  print("Berikut artikel dengan nilai cosine similarity tertinggi: ")
    # Convert the query become a vector
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
    # Calculate the similarity
  for i in range(10):
     sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
    
    # Sort the values 
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
    # Print the articles and their similarity values
  for k, v in sim_sorted:
    if v != 0.0:
      print("Nilai Similaritas:", v)
      print(documents_clean[k])        
      print()
  # Add The Query


q1 = input(str)
  # Call the function
get_similar_articles(q1, df)

query: absen
Berikut artikel dengan nilai cosine similarity tertinggi: 
Nilai Similaritas: 0.03803504597824992
kompas com agen karim benzema karim djaziri mengungkit kembali kontroversi seputar masalah yang dihadapi kliennya sehingga dipaksa meninggalkan timnas perancis dan gagal ikut piala dunia sang agen mengunggah gambar yang membuktikan karim benzema seharusnya masih bisa berperan membantu perancis dalam putaran final turnamen empat tahunan tersebut yang sudah rampung bergulir pada desember ada banyak rumor komentar dan laporan tentang cara benzema dipulangkan dari kamp perancis sebelum kick off piala dunia baca juga drama benzema unfollow pemain timnas perancis kecuali sosok ini awalnya benzema cedera saat latihan bersama tim kemudian striker real madrid tersebut diklaim sulit pulih dalam waktu singkat sehingga pelatih didier deschamps memutuskan memulangkan benzema kini djaziri meluruskan tentang cedera yang dialami benzema dia menyebut cedera peraih ballon d or tersebut tidak pa